In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.estimator.export.export_output import PredictOutput

tf.logging.set_verbosity(tf.logging.INFO)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1],name="input")

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    export_outputs={"SIGNATURE_NAME": PredictOutput({
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="final_result")
    })}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions,export_outputs=export_outputs)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./logdir")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff74c3b6e50>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './logdir', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [5]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "final_result"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [6]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=50,
    num_epochs=2,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./logdir/model.ckpt.
INFO:tensorflow:probabilities = [[0.09494176 0.09873858 0.09039416 0.10583942 0.11256394 0.10168834
  0.09588553 0.09577452 0.09882972 0.10534398]
 [0.09732568 0.10159139 0.09378022 0.10812474 0.11304431 0.10086996
  0.08458915 0.0937424  0.10277361 0.10415845]
 [0.09346767 0.10313287 0.09699849 0.10650746 0.1111665  0.1033497
  0.09465139 0.09333915 0.1026581  0.09472869]
 [0.09243365 0.09332886 0.0951745  0.11952133 0.11762298 0.09188104
  0.08977671 0.09949306 0.09726446 0.10350338]
 [0.09769514 0.09835468 0.1027671  0.10573816 0.11571977 0.09084812
  0.096601   0.09743514 0.0976697  0.09717114]
 [0.09145431 0.10569397 0.1032806  0.11748395 0.10885845 0.0945866
  0.09130552 0.08785814

INFO:tensorflow:global_step/sec: 126.551
INFO:tensorflow:probabilities = [[0.10226758 0.09232204 0.09383755 0.11939676 0.10932985 0.08744551
  0.0909941  0.09819563 0.10236522 0.10384575]
 [0.10724735 0.10161115 0.08508159 0.12011009 0.10234559 0.09413885
  0.1005706  0.09795775 0.09949388 0.0914432 ]
 [0.10807786 0.09030338 0.08710661 0.1099259  0.09871551 0.10035589
  0.09631167 0.09840851 0.10299288 0.10780181]
 [0.08867192 0.09639776 0.09573966 0.12746209 0.11641217 0.09463486
  0.10394991 0.09115455 0.09186702 0.09371004]
 [0.10054258 0.09242462 0.10329709 0.1012015  0.11026642 0.10230062
  0.09184645 0.09245408 0.10804716 0.09761942]
 [0.10028802 0.09391253 0.09236284 0.11190021 0.11475514 0.09776109
  0.09421753 0.09685849 0.10023955 0.09770463]
 [0.08323742 0.10384925 0.10428239 0.10739779 0.11788    0.09452367
  0.09355836 0.09209044 0.09436683 0.10881379]
 [0.09369545 0.10095228 0.10179084 0.11302287 0.09913065 0.0818316
  0.10065952 0.10268622 0.09179191 0.11443865]
 [0.0961

INFO:tensorflow:global_step/sec: 155.342
INFO:tensorflow:probabilities = [[0.10534387 0.09802465 0.08837668 0.09769289 0.10038717 0.10514086
  0.10073271 0.09875252 0.10962798 0.09592061]
 [0.11103088 0.08981032 0.1031829  0.09966609 0.09680413 0.08818145
  0.1034794  0.10518633 0.10821838 0.0944401 ]
 [0.10056887 0.0948827  0.09401824 0.10120162 0.09998488 0.10115308
  0.09824118 0.09370261 0.11952215 0.0967246 ]
 [0.09966387 0.09514389 0.09128809 0.10680027 0.09985298 0.10022477
  0.09703221 0.09703102 0.11107491 0.10188796]
 [0.09225886 0.09351999 0.08608184 0.11391338 0.13015294 0.09818776
  0.09263816 0.09375022 0.09922695 0.10026988]
 [0.09497919 0.09689762 0.09840909 0.10207618 0.11078437 0.09414177
  0.09820707 0.10051953 0.1123044  0.09168077]
 [0.09727069 0.09503332 0.09872495 0.10813141 0.10282054 0.09075279
  0.10074179 0.09633631 0.10608865 0.10409952]
 [0.0925239  0.10508896 0.09766212 0.10606766 0.10236288 0.09224085
  0.10279332 0.09941414 0.09914818 0.10269804]
 [0.094

INFO:tensorflow:global_step/sec: 144.464
INFO:tensorflow:probabilities = [[0.0883527  0.09512239 0.09910409 0.10080667 0.10238241 0.0990008
  0.09800354 0.10038079 0.10920493 0.1076417 ]
 [0.09506908 0.09044746 0.09360496 0.12337962 0.11751726 0.08767387
  0.09923954 0.09660306 0.10068109 0.0957841 ]
 [0.09600756 0.09440155 0.09068244 0.11753684 0.10702095 0.0891356
  0.10756342 0.09024361 0.10106472 0.10634332]
 [0.09830965 0.10115527 0.09959582 0.11061907 0.10138542 0.09510021
  0.09099159 0.09078142 0.11595191 0.09610968]
 [0.09862107 0.0914409  0.09886908 0.11109279 0.11565565 0.09628567
  0.09222797 0.09315239 0.10541145 0.09724302]
 [0.08887583 0.10024718 0.10585558 0.09773665 0.1092898  0.09238325
  0.09837437 0.08471476 0.10881875 0.11370382]
 [0.09495035 0.09099911 0.0888396  0.12334981 0.11037949 0.08877737
  0.09737995 0.09772893 0.10912363 0.09847169]
 [0.09402195 0.10388519 0.0984605  0.10861384 0.08551423 0.1013936
  0.1049054  0.08587747 0.10555396 0.1117739 ]
 [0.095078

INFO:tensorflow:global_step/sec: 154.832
INFO:tensorflow:probabilities = [[0.08791061 0.11048971 0.09926877 0.10275654 0.10358649 0.09884484
  0.10351988 0.09331949 0.10732386 0.09297988]
 [0.09272974 0.0969299  0.09572382 0.12651159 0.10232963 0.09504125
  0.09092718 0.07474444 0.1170226  0.10803986]
 [0.10676315 0.09608756 0.08420742 0.10964267 0.10133482 0.09289345
  0.09794695 0.09430637 0.10727122 0.10954647]
 [0.10115241 0.08347934 0.09823307 0.11106644 0.1147576  0.10336476
  0.10223553 0.08577443 0.09606712 0.10386935]
 [0.0887545  0.10033404 0.09672765 0.09953503 0.10898258 0.09608001
  0.09852021 0.09418019 0.1103237  0.10656219]
 [0.08221121 0.09076275 0.09667638 0.10741409 0.10504171 0.0930876
  0.09829365 0.12117202 0.10840204 0.0969385 ]
 [0.0970844  0.0885639  0.10109528 0.10708429 0.10977215 0.10000088
  0.10966192 0.08833603 0.10291672 0.09548436]
 [0.08645059 0.10719013 0.10153142 0.09458169 0.1056079  0.1096279
  0.09862986 0.10139469 0.10230345 0.09268238]
 [0.10334

...

INFO:tensorflow:global_step/sec: 168.269
INFO:tensorflow:probabilities = [[0.04382494 0.00445207 0.00552012 0.04050505 0.02939913 0.02434229
  0.00420948 0.77759033 0.02793898 0.0422175 ]
 [0.08139206 0.01258994 0.12435056 0.02806082 0.2698373  0.06978048
  0.17691056 0.00877514 0.03479448 0.19350861]
 [0.06509808 0.04807654 0.08958498 0.03868189 0.06249299 0.08340997
  0.45585057 0.01699288 0.09811333 0.04169865]
 [0.00504623 0.11611453 0.00983012 0.08918937 0.03992192 0.144749
  0.02262502 0.11407055 0.19376564 0.2646877 ]
 [0.03193794 0.01734006 0.1371711  0.5019888  0.01957126 0.08590801
  0.09350424 0.04298656 0.04457683 0.02501516]
 [0.02976572 0.02721121 0.06030691 0.00983881 0.31516257 0.0867285
  0.09468073 0.07684236 0.14762558 0.1518376 ]
 [0.12431548 0.01269448 0.05290624 0.46634427 0.01110328 0.15246242
  0.03571944 0.0910378  0.02817879 0.02523787]
 [0.04078673 0.01732078 0.00925045 0.0459029  0.04218867 0.03249635
  0.00545343 0.65712297 0.05650644 0.0929712 ]
 [0.091813

INFO:tensorflow:global_step/sec: 161.041
INFO:tensorflow:probabilities = [[0.09607244 0.00872497 0.19593869 0.36759236 0.01834125 0.169364
  0.03801307 0.03811364 0.03546193 0.03237772]
 [0.01546141 0.03093735 0.12233857 0.05494161 0.00496081 0.06722248
  0.01288415 0.01031293 0.67279196 0.00814881]
 [0.01276256 0.01947223 0.06425246 0.03890196 0.43087718 0.04117249
  0.10783347 0.0337843  0.06846502 0.18247841]
 [0.0225075  0.00227866 0.05001813 0.07118627 0.5883715  0.03432041
  0.03797241 0.0492349  0.00638743 0.13772282]
 [0.01424976 0.03448271 0.01212645 0.02353978 0.05984332 0.10133963
  0.0039786  0.3144336  0.20008616 0.23591998]
 [0.0096512  0.05259269 0.01147589 0.0799595  0.25497717 0.12024628
  0.01847047 0.08619992 0.18807389 0.17835298]
 [0.01142074 0.02595982 0.01451551 0.01141778 0.443126   0.15880568
  0.048504   0.01398043 0.0581558  0.21411426]
 [0.00215773 0.36716312 0.07855276 0.0943554  0.01879082 0.04672045
  0.20076968 0.00845367 0.15300044 0.03003592]
 [0.00637

INFO:tensorflow:global_step/sec: 157.959
INFO:tensorflow:probabilities = [[0.00998811 0.01723088 0.09773384 0.04421531 0.03499648 0.01910821
  0.02308508 0.6562429  0.05330537 0.04409387]
 [0.03983572 0.00197976 0.02756281 0.68404526 0.00200764 0.12419768
  0.03507772 0.00726856 0.07384036 0.00418451]
 [0.11401721 0.00176673 0.2512158  0.02482381 0.29469544 0.05364263
  0.16403297 0.02058451 0.04598192 0.02923908]
 [0.02899973 0.03761994 0.1361668  0.19352081 0.00724354 0.21812855
  0.12940292 0.06912797 0.16412759 0.0156621 ]
 [0.06906601 0.01064144 0.21902315 0.15358444 0.02748418 0.11558098
  0.06294599 0.02505546 0.2246278  0.09199051]
 [0.03396188 0.00384257 0.06563251 0.02550987 0.03636211 0.09468715
  0.0433064  0.06355996 0.60722923 0.02590833]
 [0.00717031 0.00292935 0.03068884 0.00805525 0.02132156 0.01220618
  0.90014404 0.00098339 0.00719639 0.00930477]
 [0.00292034 0.01523972 0.02279846 0.13900098 0.00327647 0.24121542
  0.03445033 0.09520683 0.24560691 0.20028456]
 [0.004

INFO:tensorflow:global_step/sec: 157.325
INFO:tensorflow:probabilities = [[0.00136062 0.12387967 0.00340748 0.01869088 0.21255149 0.06853054
  0.00717423 0.02192687 0.3307469  0.21173134]
 [0.00370742 0.01706122 0.0063106  0.0288986  0.14785488 0.0597918
  0.01618281 0.03798276 0.03727378 0.64493614]
 [0.00102539 0.03844719 0.00562818 0.02890273 0.03500918 0.14423193
  0.00304117 0.08058777 0.43803203 0.22509445]
 [0.01990878 0.00588718 0.00993696 0.02578737 0.0497759  0.04415744
  0.01261264 0.56686956 0.03236017 0.23270403]
 [0.25108042 0.00169383 0.01657564 0.05507203 0.01912713 0.4511342
  0.1133175  0.00419499 0.03608171 0.05172253]
 [0.01256998 0.0277879  0.00662682 0.0081922  0.25400186 0.0762074
  0.0142182  0.08124183 0.20619762 0.31295618]
 [0.01723254 0.01399514 0.03197954 0.13186844 0.00265702 0.01773136
  0.00295645 0.54639816 0.1951046  0.04007678]
 [0.01260841 0.10307002 0.06368975 0.16396457 0.05029664 0.13480672
  0.15610375 0.05590892 0.13748932 0.12206187]
 [0.003823

INFO:tensorflow:Saving checkpoints for 2200 into ./logdir/model.ckpt.
INFO:tensorflow:Loss for final step: 0.6462923.


In [7]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-16-13:17:38
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logdir/model.ckpt-2200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-16-13:17:39
INFO:tensorflow:Saving dict for global step 2200: accuracy = 0.8514, global_step = 2200, loss = 0.62837857
{'loss': 0.62837857, 'global_step': 2200, 'accuracy': 0.8514}


In [9]:
# Export the model to saved_model.pb file
def serving_input_receiver():
    inputs = {"x": tf.placeholder(dtype=tf.float32, shape=[None,28,28,1])}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

export_dir = mnist_classifier.export_savedmodel(
    export_dir_base = "./logdir",
    serving_input_receiver_fn = serving_input_receiver,
    as_text=False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default', 'SIGNATURE_NAME']
INFO:tensorflow:Restoring parameters from ./logdir/model.ckpt-2200
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./logdir/temp-1526476858/saved_model.pb
